In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib.font_manager as font_manager
import matplotlib as mpl
from matplotlib import rcParams

# Definimos path
path = '/home/daniufundar/Documents/Fundar/graficos-indice-desigualdad-genero/'

# Definimos fuente para los graficos
font_dir = [path + 'basemaps/Archivo/']
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

rcParams['font.family'] = 'Archivo'

# Definimos colores fundar
fundar_colores_primarios = ['#7BB5C4', '#9FC1AD', '#D3D3E0',  '#8d9bff', '#FF9750', '#FFD900',]
fundar_colores_secundarios = ['#B5E0EA', '#B3B3B3', '#848279', '#AFA36E', '#5D896F', '#9C9CBC', '#E27124']
fundar_white = '#F5F5F5'
fundar_black = '#151515'

In [4]:
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'

# df_people_2016 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_16.txt', delimiter=';', low_memory=False)
# df_people_2017 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_17.txt', delimiter=';', low_memory=False)
# df_people_2018 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_18.txt', delimiter=';', low_memory=False)
# df_people_2019 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_19.txt', delimiter=';', low_memory=False)
# df_people_2020 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_20.txt', delimiter=';', low_memory=False)
df_people_2021 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_21.txt', delimiter=';', low_memory=False)
df_people_2022 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_22.txt', delimiter=';', low_memory=False)

In [9]:
merged_df = pd.merge(df_people_2021[['CODUSU', 'COMPONENTE']], df_people_2022[['CODUSU', 'COMPONENTE']], on=['CODUSU', 'COMPONENTE'], how='outer', indicator=True)

# Count the number of duplicate rows
duplicated_rows = merged_df['_merge'].value_counts()['both']
print(f"Number of duplicate rows: {duplicated_rows}")

# Display the left-only rows (2021)
left_only_rows = merged_df['_merge'].value_counts()['left_only']
print(f"Left-only rows: {left_only_rows}")

# Display the right-only rows (2022)
right_only_rows = merged_df['_merge'].value_counts()['right_only']
print(f"Right-only rows: {right_only_rows}")

# Select the right-only and right observation from both rows in merged_df
right_only_rows = merged_df[merged_df['_merge'] == 'right_only']
left_only_rows = merged_df[merged_df['_merge'] == 'left_only']
both_rows = merged_df[merged_df['_merge'] == 'both']
right_both_rows = pd.concat([both_rows, right_only_rows])

# Option 1: Merge two dataframes with "copies"
df_people_2021['rel_PONDERA'] = df_people_2021['PONDERA']/df_people_2021['PONDERA'].sum()
df_people_2022['rel_PONDERA'] = df_people_2022['PONDERA']/df_people_2022['PONDERA'].sum()

df_merged_with_copies = pd.concat([df_people_2021, df_people_2022])

# Option 2: Merge two dataframes erasing the first copy (keeping right only and right both observations)
df_people_2021_no_dupl = pd.merge(left_only_rows, df_people_2021, on=["CODUSU", "COMPONENTE"], how='inner')
df_people_2022_no_dupl = pd.merge(right_both_rows, df_people_2022, on=["CODUSU", "COMPONENTE"], how='inner')

df_merged_without_copies = pd.concat([df_people_2021_no_dupl, df_people_2022_no_dupl])


# Length of resulting dataframes
n_with_copies = len(df_merged_with_copies)
n_without_copies = len(df_merged_without_copies)
print(f"La longitud de los dataframes con y sin replicas son {n_with_copies, n_without_copies}")

Number of duplicate rows: 30417
Left-only rows: 48100
Right-only rows: 47523
La longitud de los dataframes con y sin replicas son (155859, 127958)


In [62]:
#cod_provincia
dict_cod_provincia = {2: "Ciudad de Buenos Aires",
 6: "Buenos Aires",
 10: "Catamarca",
 14: "Córdoba",
 18: "Corrientes",
 22: "Chaco",
 26: "Chubut",
 30: "Entre Ríos",
 34: "Formosa",
 38: "Jujuy",
 42: "La Pampa",
 46: "La Rioja",
 50: "Mendoza",
 54: "Misiones",
 58: "Neuquén",
 62: "Río Negro",
 66: "Salta",
 70: "San Juan",
 74: "San Luis",
 78: "Santa Cruz",
 82: "Santa Fe",
 86: "Santiago del Estero",
 90: "Tucumán",
 94: "Tierra del Fuego"}

In [67]:
## Defino funciones dimension mercado de trabajo e ingresos

# Componente: Inserción laboral

# T1. 
# Variable: Actividad productiva
# Indicador: Ratio M/V en tasa de empleo

def ratio_tasa_empleo(df, tipo='Aglomerado', base='Individual'):

    """
    INPUTS
    df: DataFrame. Tabla input EPH
    tipo: string. Tipo de encuesta de la EPH, Aglomerado o Urbano. Default Aglomerado
    base: string. Tipo de base de la encuesta de la EPH, Individual u Hogar. Default Individual

    OUTPUTS
    ratio: DataFrame. Tabla con Ratios en tasa de empleo M/V desagregado por Aglomerado o Provincia
    error: DataFrame. Tabla con los errores asociados a los Ratios

    """

    # Check if the necessary columns exist in the input DataFrame
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA']):
        raise ValueError("El DataFrame de entrada debe tener las siguientes columnas: 'CH04', 'CH06', 'ESTADO', 'PONDERA'")
        
    # Check if the input value of "tipo" is valid
    if tipo not in ['Aglomerado', 'Urbano']:
        return None, 'Error: tipo debe ser "Aglomerado" o "Urbano"'
    if base not in ['Individual', 'Hogar']:
        return None, 'Error: base debe ser "Individual" o "Hogar"'
    if not isinstance(df, pd.DataFrame):
        return None, 'Error: df debe ser un DataFrame de pandas'

    if tipo=='Aglomerado':
        var = 'AGLOMERADO' 
    elif tipo=='Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')
    numerador = df_temp[df_temp['ESTADO']==1].groupby(['CH04', var])['PONDERA'].sum()       # agrupo por genero y provincia y sumo
    
    denominador = df_temp.groupby(['CH04', var])['PONDERA'].sum()
    df_tasa = numerador.div(denominador, fill_value=np.nan)
    df_tasa = df_tasa.to_frame().unstack(level=0)
    
    ratio = (df_tasa['PONDERA'][2]/df_tasa['PONDERA'][1] * 100).to_frame()
    ratio.rename(columns={0: 'Actividad productiva'}, inplace=True)

    # Calculate error

    error = pd.DataFrame(index=ratio.index)

    return ratio, error


# T2. 
# Variable: Búsqueda de trabajo
# Indicador: Ratio M/V en tasa de desempleo

def ratio_tasa_desempleo(df, tipo='Aglomerado', base='Individual'):

    """
    INPUTS
    df: DataFrame. Tabla input EPH
    tipo: string. Tipo de encuesta de la EPH, Aglomerado o Urbano. Default Aglomerado
    base: string. Tipo de base de la encuesta de la EPH, Individual u Hogar. Default Individual

    OUTPUTS
    ratio: DataFrame. Tabla con Ratios en tasa de desempleo M/V desagregado por Aglomerado o Provincia
    error: DataFrame. Tabla con los errores asociados a los Ratios

    """
    # Check if the necessary columns exist in the input DataFrame
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA']):
        raise ValueError("The input DataFrame must have the following columns: 'CH04', 'CH06', 'ESTADO', 'PONDERA'")
        
    # Check if the input value of "tipo" is valid
    if tipo not in ['Aglomerado', 'Urbano']:
        return None, 'Error: tipo debe ser "Aglomerado" o "Urbano"'
    if base not in ['Individual', 'Hogar']:
        return None, 'Error: base debe ser "Individual" o "Hogar"'
    if not isinstance(df, pd.DataFrame):
        return None, 'Error: df debe ser un DataFrame de pandas'

    if tipo=='Aglomerado':
        var = 'AGLOMERADO' 
    elif tipo=='Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')                                           
    numerador = df_temp[df_temp['ESTADO']==2].groupby(['CH04', var])['PONDERA'].sum()               # agrupo por genero y provincia y sumo
    df_estado = df_temp[(df_temp['ESTADO'] == 1) | (df_temp['ESTADO'] == 2)]                        # selecciona las filas donde la columna ESTADO tenga un valor igual a 1 o 2 (personas activas)
    denominador = df_estado.groupby(['CH04', var])['PONDERA'].sum()                                 # agrupo por genero y provincia y sumo
    df_tasa = (numerador / denominador).to_frame().unstack(level=0)                                 # calculo tasa

    ratio = (df_tasa['PONDERA'][1]/df_tasa['PONDERA'][2] * 100).to_frame()                          # el ratio es V/M para que quede elevado a la menos 1
    ratio.rename(columns={0: 'Búsqueda de trabajo'}, inplace=True)

    error = 'a definir'

    return ratio, error

# T3.
# Variable: Acceso a derechos laborales
# Indicador: Ratio M/V en tasa de empleo asalariado con descuento jubilatorio

def ratio_tasa_registro(df, tipo='Aglomerado', base='Individual'):

    """
    INPUTS
    df: DataFrame. Tabla input EPH
    tipo: string. Tipo de encuesta de la EPH, Aglomerado o Urbano. Default Aglomerado
    base: string. Tipo de base de la encuesta de la EPH, Individual u Hogar. Default Individual

    OUTPUTS
    ratio: DataFrame. Tabla con Ratios en tasa de informalidad M/V desagregado por Aglomerado o Provincia
    error: DataFrame. Tabla con los errores asociados a los Ratios

    """

    # Check if the necessary columns exist in the input DataFrame
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA', 'CAT_OCUP', 'PP07H']):
        raise ValueError("The input DataFrame must have the following columns: 'CH04', 'CH06', 'ESTADO', 'PONDERA', 'CAT_OCUP', 'PP07H'")
        
    # Check if the input value of "tipo" is valid
    if tipo not in ['Aglomerado', 'Urbano']:
        return None, 'Error: tipo debe ser "Aglomerado" o "Urbano"'
    if base not in ['Individual', 'Hogar']:
        return None, 'Error: base debe ser "Individual" o "Hogar"'
    if not isinstance(df, pd.DataFrame):
        return None, 'Error: df debe ser un DataFrame de pandas'

    if tipo=='Aglomerado':
        var = 'AGLOMERADO' 
    elif tipo=='Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')
    df_estado = df_temp[(df_temp['ESTADO']==1) & (df_temp['CAT_OCUP']==3)]                      # Población ocupada asalariada
    numerador = df_estado[df_estado['PP07H']==1].groupby(['CH04', var])['PONDERA'].sum()        # Población ocupada asalariada con descuento jubilatorio
    denominador = df_estado.groupby(['CH04', var])['PONDERA'].sum()
    df_tasa = (numerador / denominador).to_frame().unstack(level=0)                             # calculo tasa

    ratio = (df_tasa['PONDERA'][2]/df_tasa['PONDERA'][1] * 100).to_frame()                      # Calculo ratio
    ratio.rename(columns={0: 'Acceso a derechos laborales'}, inplace=True)

    error = pd.DataFrame(index=ratio.index)
    error = 'a definir'

    # for i in ratio.index:
    #     if denominador[i] == 0:
    #         error[i] = 'Warning: El denominador es cero, no se puede calcular el ratio'
    #     else:
    #         error[i] = np.sqrt((ratio[i]*(1-ratio[i])/denominador[i]))

    return ratio, error

# T4. 
# Variable: Participación en el trabajo remunerado
# Indicador: Ratio M/V en Horas semanales promedio destinadas al trabajo remunerado

def ratio_tasa_horas_remuneradas(df, tipo='Aglomerado', base='Individual'):

    """
    INPUTS
    df: DataFrame. Tabla input EPH
    tipo: string. Tipo de encuesta de la EPH, Aglomerado o Urbano. Default Aglomerado
    base: string. Tipo de base de la encuesta de la EPH, Individual u Hogar. Default Individual

    OUTPUTS
    ratio: DataFrame. Tabla con Ratios en tasa de horas remuneradas M/V desagregado por Aglomerado o Provincia
    error: DataFrame. Tabla con los errores asociados a los Ratios

    """

    # Check if the necessary columns exist in the input DataFrame       
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA', 'CAT_OCUP', 'PP07H']):
        raise ValueError("The input DataFrame must have the following columns: 'CH04', 'CH06', 'ESTADO', 'PONDERA', 'CAT_OCUP', 'PP07H'")
        
    # Check if the input value of "tipo" is valid
    if tipo not in ['Aglomerado', 'Urbano']:
        return None, 'Error: tipo debe ser "Aglomerado" o "Urbano"'
    if base not in ['Individual', 'Hogar']:
        return None, 'Error: base debe ser "Individual" o "Hogar"'
    if not isinstance(df, pd.DataFrame):
        return None, 'Error: df debe ser un DataFrame de pandas'

    if tipo=='Aglomerado':
        var = 'AGLOMERADO' 
    elif tipo=='Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')
    df_temp['PP3E_TOT'] = pd.to_numeric(df_temp['PP3E_TOT'], errors='coerce') # coerce invalid parsing to NaN

    df_estado = df_temp[(df_temp['ESTADO']==1) & (df_temp['PP3E_TOT']>0) & (df_temp['PP3E_TOT']!=999)]  
    df_tasa = df_estado.groupby(['CH04', var])['PONDERA'].mean().to_frame().unstack(level=0)
    
    ratio = (df_tasa['PONDERA'][2]/df_tasa['PONDERA'][1] * 100).to_frame()
    ratio.rename(columns={0: 'Participación en el trabajo remunerado'}, inplace=True)
    
    error = 'a definir'

    return ratio, error


# T5. 
# Variable: Horas trabajadas en el hogar
# Indicador: Ratio M/V en Horas semanales promedio destinadas al TNR en el hogar

        # en funcion de la enut


# Componente: Autonomía económica

# T6. 
# Variable: Dependencia económica 
# Indicador: Ratio M/V en Población inactiva y que no estudia sin ingresos propios

def ratio_tasa_poblacion_dependiente(df, tipo='Aglomerado', base='Individual'):

    # En este caso dividir el indicador segun algun percentil de ingresos. Por ejemplo de 90 a 60. Ver literatura al respecto 

    """
    INPUTS
    df: DataFrame. Tabla input EPH
    tipo: string. Tipo de encuesta de la EPH, Aglomerado o Urbano. Default Aglomerado
    base: string. Tipo de base de la encuesta de la EPH, Individual u Hogar. Default Individual

    OUTPUTS
    ratio: DataFrame. Tabla con Ratios en tasa M/V de población inactiva que no estudia y no tiene ingresos propios, desagregado por Aglomerado o Provincia
    error: DataFrame. Tabla con los errores asociados a los Ratios

    """

    # Check if the necessary columns exist in the input DataFrame
    if not all(col in df.columns for col in ['CH04', 'CH06', 'ESTADO', 'PONDERA', 'P47T', 'CH10', 'PONDII']):
        raise ValueError("The input DataFrame must have the following columns: 'CH04', 'CH06', 'ESTADO', 'PONDERA', 'P47T', 'CH10', 'PONDII'")
        
    # Check if the input value of "tipo" is valid
    if tipo not in ['Aglomerado', 'Urbano']:
        return None, 'Error: tipo debe ser "Aglomerado" o "Urbano"'
    if base not in ['Individual', 'Hogar']:
        return None, 'Error: base debe ser "Individual" o "Hogar"'
    if not isinstance(df, pd.DataFrame):
        return None, 'Error: df debe ser un DataFrame de pandas'
    
    if tipo=='Aglomerado':
        var = 'AGLOMERADO' 
    elif tipo=='Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')

    numerador = df_temp[(df_temp['ESTADO']==3) & (df_temp['P47T']==0) & (df_temp['CAT_INAC']!=3)].groupby(['CH04', var])['PONDII'].sum()
    denominador = df_temp.groupby(['CH04', var])['PONDERA'].sum()
    df_tasa = (numerador / denominador).to_frame().unstack(level=0)                         # calculo tasa

    ratio = (df_tasa[0][2]/df_tasa[0][1] * 100).to_frame()
    ratio.rename(columns={0: 'Dependencia económica'}, inplace=True)
    
    error = pd.DataFrame(index=ratio.index)
    error = 'a definir'


    return ratio, error

# T7.                                   # TODO: ajustar con IPC
# Variable: Ingreso salarial
# Indicador: Ratio M/V en Ingreso salarial promedio mensual

def ratio_ingreso_laboral_mensual(df, tipo='Aglomerado', base='Individual'):

    """
    INPUTS
    df: DataFrame. Tabla input EPH
    tipo: string. Tipo de encuesta de la EPH, Aglomerado o Urbano. Default Aglomerado
    base: string. Tipo de base de la encuesta de la EPH, Individual u Hogar. Default Individual

    OUTPUTS
    ratio: DataFrame. Tabla con Ratios del ingreso individual mensual laboral de M/V, desagregado por Aglomerado o Provincia
    error: DataFrame. Tabla con los errores asociados a los Ratios

    """

    if tipo=='Aglomerado':
        var = 'AGLOMERADO' 
    elif tipo=='Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')
    df_estado = df_temp[(df_temp['ESTADO'] == 1) & (~df_temp['CAT_OCUP'].isin([1, 4, 9]))]

    df_ingreso = df_estado[(df_estado['P21']>0) & (df_estado['P21']!=-9)].sum()
    df_tasa = df_ingreso.groupby(['CH04', var])['PONDIIO'].median().to_frame().unstack(level=0)  
    
    ratio = (df_tasa['PONDIIO'][2]/df_tasa['PONDIIO'][1] * 100).to_frame()
    ratio.rename(columns={0: 'Ingreso laboral'}, inplace=True)
    
    error = 'a definir'

    return ratio, error


# T8.                               # TODO: ver definicion monoparentalidad
# Variable: Pobreza
# Indicador: Ratio M/V en Población con ingreso total individual inferior a LP

def ratio_pobreza(df, CBT, tipo='Aglomerado', base='Individual'):

    """
    INPUTS
    df: DataFrame. Tabla input EPH
    tipo: string. Tipo de encuesta de la EPH, Aglomerado o Urbano. Default Aglomerado
    base: string. Tipo de base de la encuesta de la EPH, Individual u Hogar. Default Individual

    OUTPUTS
    ratio: DataFrame. Tabla con Ratios del ingreso individual mensual laboral de M/V, desagregado por Aglomerado o Provincia
    error: DataFrame. Tabla con los errores asociados a los Ratios

    """


    if tipo=='Aglomerado':
        var = 'AGLOMERADO' 
    elif tipo=='Urbano':
        var = 'PROVINCIA'

    df_temp = df.query('CH06 >= 16 & CH06 < 65')
    df_estado = df_temp[(df_temp['ESTADO'] == 1) & (~df_temp['CAT_OCUP'].isin([1, 4, 9]))]

    numerador = df_estado[(df_estado['P21']>0) & (df_estado['P21']!=-9)].sum()
    denominador = 'TBD'
    df_tasa = df_estado.groupby(['CH04', var])['PONDIIO'].mean().to_frame().unstack(level=0)  
    
    ratio = (df_tasa['PONDIIO'][2]/df_tasa['PONDIIO'][1] * 100).to_frame()
    ratio.rename(columns={0: 'Ingreso laboral'}, inplace=True)
    
    error = 'a definir'

    return ratio, error



def df_merge_ratios(ratio_empleo, ratio_desempleo, ratio_informalidad, ratio_horas_remuneradas, 
                ratio_poblacion_dependiente, ratio_ingreso_laboral, ratio_pobreza, diccionario):

    df_ratio = pd.concat([ratio_empleo, ratio_desempleo, ratio_informalidad, ratio_horas_remuneradas, 
                    ratio_poblacion_dependiente, ratio_ingreso_laboral, ratio_pobreza], axis=1)
                    
    df_ratio['Nombre'] = df_ratio.index.map(diccionario)

    df_ratio = df_ratio.set_index('Nombre')
    
    return df_ratio

In [66]:
ratio_empleo, error_empleo = ratio_tasa_empleo(df_people_2021, tipo='Urbano', base='Individual')
ratio_desempleo, error_desempleo = ratio_tasa_desempleo(df_people_2021, tipo='Urbano', base='Individual')
ratio_informalidad, error_informalidad = ratio_tasa_registro(df_people_2021, tipo='Urbano', base='Individual')
ratio_horas_remuneradas, error_horas_remuneradas = ratio_tasa_horas_remuneradas(df_people_2021, tipo='Urbano', base='Individual')
ratio_poblacion_dependiente, error_poblacion_dependiente = ratio_tasa_poblacion_dependiente(df_people_2021, tipo='Urbano', base='Individual')
ratio_ingreso_laboral, error_ingreso_laboral = ratio_tasa_empleo(df_people_2021, tipo='Urbano', base='Individual')
ratio_pobreza, error_pobreza = ratio_tasa_empleo(df_people_2021, tipo='Urbano', base='Individual')

df_ratio_2021 = df_merge_ratios(ratio_empleo, ratio_desempleo, ratio_informalidad, ratio_horas_remuneradas, 
                ratio_poblacion_dependiente, ratio_ingreso_laboral, ratio_pobreza, dict_cod_provincia)

48
48
48
48
48


In [65]:
df_ratio_2021.iloc[:,0:5]

,Actividad productiva,Búsqueda de trabajo,Acceso a derechos laborales,Participación en el trabajo remunerado,Dependencia económica
Nombre,,,,,
Ciudad de Buenos Aires,92.127449,143.408430,94.378673,104.615024,159.391859
Buenos Aires,71.676538,76.737452,92.330813,92.303697,372.011704
Catamarca,74.809933,62.304473,104.334782,103.003372,178.102083
Córdoba,68.596462,53.736201,90.888590,89.369581,385.303643
Corrientes,71.753725,190.514579,97.700547,104.433645,160.778827
Chaco,64.369014,103.363918,124.160572,89.218701,219.355868
Chubut,73.368752,131.096370,91.600942,88.424742,512.356275
Entre Ríos,71.786217,57.776366,92.638600,92.835137,418.150114
Formosa,51.901224,220.595458,108.371130,100.375501,178.176566


In [ ]:
ratio_empleo, error_empleo = ratio_tasa_empleo(df_people_2022, tipo='Urbano', base='Individual')
ratio_desempleo, error_desempleo = ratio_tasa_desempleo(df_people_2022, tipo='Urbano', base='Individual')
ratio_informalidad, error_informalidad = ratio_tasa_registro(df_people_2022, tipo='Urbano', base='Individual')
ratio_horas_remuneradas, error_horas_remuneradas = ratio_tasa_horas_remuneradas(df_people_2022, tipo='Urbano', base='Individual')
ratio_poblacion_dependiente, error_poblacion_dependiente = ratio_tasa_poblacion_dependiente(df_people_2022, tipo='Urbano', base='Individual')
ratio_ingreso_laboral, error_ingreso_laboral = ratio_tasa_empleo(df_people_2022, tipo='Urbano', base='Individual')
ratio_pobreza, error_pobreza = ratio_tasa_empleo(df_people_2022, tipo='Urbano', base='Individual')

df_ratio_2022 = df_merge_ratios(ratio_empleo, ratio_desempleo, ratio_informalidad, ratio_horas_remuneradas, 
                ratio_poblacion_dependiente, ratio_ingreso_laboral, ratio_pobreza, dict_cod_provincia)

In [ ]:
df_ratio_2022.iloc[:,0:5]

In [ ]:
ratio_empleo, error_empleo = ratio_tasa_empleo(df_merged_with_copies, tipo='Urbano', base='Individual')
ratio_desempleo, error_desempleo = ratio_tasa_desempleo(df_merged_with_copies, tipo='Urbano', base='Individual')
ratio_informalidad, error_informalidad = ratio_tasa_registro(df_merged_with_copies, tipo='Urbano', base='Individual')
ratio_horas_remuneradas, error_horas_remuneradas = ratio_tasa_horas_remuneradas(df_merged_with_copies, tipo='Urbano', base='Individual')
ratio_poblacion_dependiente, error_poblacion_dependiente = ratio_tasa_poblacion_dependiente(df_merged_with_copies, tipo='Urbano', base='Individual')
ratio_ingreso_laboral, error_ingreso_laboral = ratio_tasa_empleo(df_merged_with_copies, tipo='Urbano', base='Individual')
ratio_pobreza, error_pobreza = ratio_tasa_empleo(df_merged_with_copies, tipo='Urbano', base='Individual')

df_ratio_with_copies = df_merge_ratios(ratio_empleo, ratio_desempleo, ratio_informalidad, ratio_horas_remuneradas, 
                ratio_poblacion_dependiente, ratio_ingreso_laboral, ratio_pobreza, dict_cod_provincia)

In [ ]:
df_ratio_with_copies.iloc[:,0:5]

In [ ]:
ratio_empleo, error_empleo = ratio_tasa_empleo(df_merged_without_copies, tipo='Urbano', base='Individual')
ratio_desempleo, error_desempleo = ratio_tasa_desempleo(df_merged_without_copies, tipo='Urbano', base='Individual')
ratio_informalidad, error_informalidad = ratio_tasa_registro(df_merged_without_copies, tipo='Urbano', base='Individual')
ratio_horas_remuneradas, error_horas_remuneradas = ratio_tasa_horas_remuneradas(df_merged_without_copies, tipo='Urbano', base='Individual')
ratio_poblacion_dependiente, error_poblacion_dependiente = ratio_tasa_poblacion_dependiente(df_merged_without_copies, tipo='Urbano', base='Individual')
ratio_ingreso_laboral, error_ingreso_laboral = ratio_tasa_empleo(df_merged_without_copies, tipo='Urbano', base='Individual')
ratio_pobreza, error_pobreza = ratio_tasa_empleo(df_merged_without_copies, tipo='Urbano', base='Individual')

df_ratio_without_copies = df_merge_ratios(ratio_empleo, ratio_desempleo, ratio_informalidad, ratio_horas_remuneradas, 
                ratio_poblacion_dependiente, ratio_ingreso_laboral, ratio_pobreza, dict_cod_provincia)

In [ ]:
df_ratio_without_copies.iloc[:,0:5]

In [ ]:
comparacion_indicadores = pd.DataFrame([df_ratio_2021['Actividad productiva'].iloc[:,0], 
        df_ratio_2022['Actividad productiva'].iloc[:,0], 
        df_ratio_with_copies['Actividad productiva'].iloc[:,0],
        df_ratio_without_copies['Actividad productiva'].iloc[:,0]])
comparacion_indicadores

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax = plt.axes([0.05, 0.05, 0.85, 0.85])
ax.plot(comparacion_indicadores.columns, comparacion_indicadores.T.values, marker='.', markersize=10, lw=0.2, label=['2021', '2022', 'copies', 'no copies'])
ax.legend()
for label in ax.get_xticklabels():
  label.set_rotation(45)
  label.set_ha('right')

In [ ]:
comparacion_indicadores = pd.DataFrame([df_ratio_2021['Acceso a derechos laborales'], 
        df_ratio_2022['Acceso a derechos laborales'], 
        df_ratio_with_copies['Acceso a derechos laborales'],
        df_ratio_without_copies['Acceso a derechos laborales']])
comparacion_indicadores

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax = plt.axes([0.05, 0.05, 0.85, 0.85])
ax.plot(comparacion_indicadores.columns, comparacion_indicadores.T.values, marker='.', markersize=10, lw=0.2, label=['2021', '2022', 'copies', 'no copies'])
ax.legend()
ax.grid(lw=.5)
ax.axhline(y=100)
for label in ax.get_xticklabels():
  label.set_rotation(45)
  label.set_ha('right')

In [ ]:
comparacion_indicadores = pd.DataFrame([df_ratio_2021['Búsqueda de trabajo'], 
        df_ratio_2022['Búsqueda de trabajo'], 
        df_ratio_with_copies['Búsqueda de trabajo'],
        df_ratio_without_copies['Búsqueda de trabajo']])
comparacion_indicadores

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax = plt.axes([0.05, 0.05, 0.85, 0.85])
ax.plot(comparacion_indicadores.columns, comparacion_indicadores.T.values, marker='.', markersize=10, lw=0.2, label=['2021', '2022', 'copies', 'no copies'])
ax.legend()
ax.grid(lw=.5)
ax.axhline(y=100)
ax.set_ylim([50,150])
for label in ax.get_xticklabels():
  label.set_rotation(45)
  label.set_ha('right')

In [ ]:
comparacion_indicadores = pd.DataFrame([df_ratio_2021['Participación en el trabajo remunerado'], 
        df_ratio_2022['Participación en el trabajo remunerado'], 
        df_ratio_with_copies['Participación en el trabajo remunerado'],
        df_ratio_without_copies['Participación en el trabajo remunerado']])
comparacion_indicadores

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax = plt.axes([0.05, 0.05, 0.85, 0.85])
ax.plot(comparacion_indicadores.columns, comparacion_indicadores.T.values, marker='.', markersize=10, lw=0.2, label=['2021', '2022', 'copies', 'no copies'])
ax.legend()
ax.grid(lw=.5)
ax.axhline(y=100)
#ax.set_ylim([50,150])
for label in ax.get_xticklabels():
  label.set_rotation(45)
  label.set_ha('right')